In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import pprint

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# 书上题目5.1
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

In [3]:
datasets, labels = create_data()
train_data = pd.DataFrame(datasets, columns=labels)
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [5]:
# 数据集D的经验熵H(D)
def entropy(datasets, class_col=-1):
    # 样本数量
    data_len = len(datasets)
    # 类别列
    class_col_data = [data[class_col] for data in datasets]
    # 类别集合
    class_set = set(class_col_data)
    # 每个类别的样本数量
    class_count = {c : class_col_data.count(c) for c in class_set}
    # 每个类别的率
    class_probs = {c : class_count[c] / data_len for c in class_set}
    # 经验熵
    ent = -sum([prob * math.log(prob, 2) for c, prob in class_probs.items()])

    return ent

In [7]:
# 特征A对数据集D的经验条件熵H(D|A)
def cond_entropy(datasets, feature_col):
    # 样本数量
    data_len = len(datasets)
    # 特征A的类别集合
    feature_set = set([data[feature_col] for data in datasets])
    # 特征A的每个类别对应的样本集
    feature_split_data = {c : [data for data in datasets if data[feature_col] == c] for c in feature_set}
    # 经验条件熵
    cond_ent = sum([len(datas) / data_len * entropy(datas) for c, datas in feature_split_data.items()])
    
    return cond_ent

In [8]:
# 训练数据集D关于特征A的值的熵HA(D)
def feature_entropy(datasets, feature_col):
    # 样本数量
    data_len = len(datasets)
    # 特征A的一列数据
    feature_data = [data[feature_col] for data in datasets]
    # 特征A的类别集合
    feature_set = set(feature_data)
    # 特征A的每个类别的样本数占总全样本数之比
    feature_count_radio = [feature_data.Count(value) / data_len for value in feature_set]
    
    ent = -sum([ratio * math.log(ratio, 2) for ratio in feature_count_radio])
    return ent

In [15]:
def choose_best_col_id3(datasets):
    # 样本维度
    feature_count = len(datasets[0])
    # 经验熵
    ent = entropy(datasets)
    # 各个特征的信息增益
    info_gains = []

    for feature_col in range(feature_count - 1):    # 最后一维是类别
        cond_ent = cond_entropy(datasets, feature_col)
        info_gain = ent - cond_ent
        info_gains.append((feature_col, info_gain))
        print('信息增益: 特征({}) - {:.3f}'.format(labels[feature_col], info_gain))

    # 比较大小
    best_ = max(info_gains, key = lambda x : x[-1])
    return '特征({}) 的信息增益最大，选择为根节点特征'.format(labels[best_[0]])

choose_best_col_id3(np.array(datasets))

信息增益: 特征(年龄) - 0.083
信息增益: 特征(有工作) - 0.324
信息增益: 特征(有自己的房子) - 0.420
信息增益: 特征(信贷情况) - 0.363


'特征(有自己的房子) 的信息增益最大，选择为根节点特征'

In [20]:
def choose_best_col_c4dot5(datasets):
    # 样本维度
    feature_count = len(datasets[0])
    # 经验熵
    ent = entropy(datasets)
    # 各个特征的信息增益比
    info_gains_ratios = []

    for feature_col in range(feature_count - 1):    # 最后一维是类别
        cond_ent = cond_entropy(datasets, feature_col)
        feature_ent = cond_entropy(datasets, feature_col)
        info_gain_ratio = (ent - cond_ent) / feature_ent # 信息增益比
        info_gains_ratios.append((feature_col, info_gain_ratio))
        print('信息增益比: 特征({}) - {:.3f}'.format(labels[feature_col], info_gain_ratio))

    # 比较大小
    best_ = max(info_gains_ratios, key = lambda x : x[-1])
    return '特征({}) 的信息增益比最大，选择为根节点特征'.format(labels[best_[0]])

choose_best_col_c4dot5(np.array(datasets))

信息增益比: 特征(年龄) - 0.093
信息增益比: 特征(有工作) - 0.500
信息增益比: 特征(有自己的房子) - 0.762
信息增益比: 特征(信贷情况) - 0.597


'特征(有自己的房子) 的信息增益比最大，选择为根节点特征'